# Tutorial: Build Your First Question Answering System with Haystack
From the Haystack [tutorial](https://haystack.deepset.ai/tutorials/01_basic_qa_pipeline)

In [ ]:
import logging
from haystack.document_stores import InMemoryDocumentStore
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline
from haystack.nodes import BM25Retriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import FARMReader
from haystack.utils import print_answers
import pathlib

In [ ]:
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
DATA_DIR = pathlib.Path().absolute().parent / "data"

# Set up the DocumentStore

In [ ]:
document_store = InMemoryDocumentStore(use_bm25=True)

# Prepare Documents

In [ ]:
from haystack.utils import fetch_archive_from_http

doc_dir = DATA_DIR / "build_your_first_question_answering_system"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip",
    output_dir=doc_dir
)

In [ ]:
files_to_index = [(doc_dir / f) for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

# Initialize the Retriever

In [ ]:
retriever = BM25Retriever(document_store=document_store)

# Initialize the Reader

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

# Create the Retriever-Reader Pipeline

In [ ]:
pipe = ExtractiveQAPipeline(reader, retriever)

# Ask Questions

In [ ]:
prediction = pipe.run(
    query="Who is the father of Arya Stark?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

In [ ]:
print_answers(prediction, details="minimum")